# <center><font color='green' face='cursive'>Выпускной проект «Создание рекомендательной системы»</font></center>

### Шаг 1. Знакомство с датасетами

Итак, у продакт-менеджера есть идея организовать допродажу в корзине для увеличения среднего чека. Для этого нужна рекомендательная система по курсам. В ваших силах как аналитика — составить для них таблицу, где каждому ID курса будут предоставлены ещё два курса, которые будут рекомендаваться. 

<div class="alert alert-block alert-info">
<b>А13.1.1.1 Задание 1</b><br>Продажи за какие годы есть в ваших данных?</div>

<pre>
SELECT
    DISTINCT EXTRACT
        (
            YEAR
        FROM
            c.purchased_at
        )
FROM    
    final.carts c
</pre>

<div class="alert alert-block alert-success"><ul><li>2017</li><li>2018</li></ul></div>

### Шаг 2. Подготовка данных

Для построения рекомендательной системы вы решили определить, какие другие курсы чаще всего покупаются вместе с конкретным курсом (необязательно в рамках одной корзины). Так вы сможете ранжировать все курсы и взять только те, что заинтересуют клиентов с наибольшей вероятностью.

На этом этапе вам нужно подготовить данные для анализа: написать SQL-запрос, который выгружает данные по всем продажам курсов в разрезе пользователей.

<div class="alert alert-block alert-info">
<b>А13.1.1.2 Задание 2</b><br>Сколько клиентов покупали курсы?</div>

<pre>
SELECT
    COUNT(DISTINCT c.user_id)
FROM
    final.carts AS c
JOIN
    final.cart_items AS ci
        ON
            c.id = ci.cart_id
WHERE
    c.state = 'successful' AND ci.resource_type = 'Course'
</pre>

<div class="alert alert-block alert-success">49006</div>

<div class="alert alert-block alert-info">Сколько всего есть различных курсов?</div>

<pre>
SELECT
   COUNT(DISTINCT ci.resource_id)
FROM
    final.cart_items ci
WHERE
    ci.resource_type = 'Course'
</pre>

<div class="alert alert-block alert-success">127</div>

<div class="alert alert-block alert-info">Каково среднее число купленных курсов на одного клиента? Округлите до двух знаков после точки-разделителя.</div>

<pre>
WITH purchase AS (
    SELECT
        COUNT(ci.resource_id)::numeric course_count,
        COUNT(DISTINCT c.user_id)::numeric clients_count
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
)
SELECT
    ROUND(((p.course_count / p.clients_count)),2) AS count_courses_per_user
FROM
    purchase AS p
</pre>

<div class="alert alert-block alert-success">1.44</div>

<div class="alert alert-block alert-info">Сколько клиентов купили больше одного курса?</div>

<pre>
WITH more_one AS (
    SELECT
        c.user_id users,
        COUNT(ci.resource_id) course_count
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    GROUP by
        c.user_id
    HAVING
        COUNT(DISTINCT ci.resource_id) > 1
)
SELECT
    COUNT(mo.users) AS users_more_one_courses
FROM
    more_one AS mo
</pre>

<div class="alert alert-block alert-success">12656</div>

### Шаг 3. Обработка данных

Итак, вы получили файл с данными по продажам курсов в разрезе пользователей (по купившим больше одного курса). Перейдем к его анализу в Python. Вам необходимо разбить все покупки курсов на пары, после чего ранжировать их для каждого курса.

<div class="alert alert-block alert-info">
<b>А13.1.1.3 Задание 3</b><br>Сколько различных пар курсов встречаются вместе в покупках клиентов?</div>

В процессе решения данной задачи был написан 1 запрос SQL для последующей выгрузки и анализа в Python.
<pre>
WITH purchases AS
    (
    SELECT
        DISTINCT c.user_id users,
        ci.resource_id,
        c.purchased_at
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    ORDER BY 
        c.user_id
    ),
    
more_one AS
    (
    SELECT
        DISTINCT c.user_id users,
        COUNT(ci.resource_id)
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    GROUP BY
        c.user_id
    HAVING
        COUNT(ci.resource_id) > 1
    ORDER BY 
        c.user_id
    )
    
SELECT
    *
FROM 
    purchases
    
WHERE EXISTS       
    (
        SELECT
            users
        FROM
            more_one
        WHERE
            purchases.users = more_one.users
    )
</pre>

Но данный запрос, как оказалось содержит **59** пользователей ([ссылка](https://drive.google.com/file/d/1iDz26cpGuD-_HRIYkKLt8RxYR8-6YN3b/view?usp=sharing)), которые купили 1 и тот же курс **2 раза** (объяснения этому я пока не нахожу, поэтому стоит эти данные исключить и переписать запрос.)

<pre>
WITH purchases AS
    (
    SELECT
        DISTINCT c.user_id,
        ci.resource_id,
        c.purchased_at
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    ORDER BY 
        c.user_id
    )
    
SELECT
    *
FROM 
    purchases
    
WHERE EXISTS       
    (
        SELECT
            user_id
        FROM
            (
                SELECT
                    DISTINCT user_id,
                    COUNT(resource_id) as course_count
                FROM
                    purchases
                GROUP BY
                    user_id
            ) AS counter
        WHERE
            course_count > 1 AND purchases.user_id = counter.user_id
    )
</pre>

или просто добавить в предидущий запрос **DISTINCT** в ту часть, где **COUNT(DISTINCT ci.resource_id) > 1**

Для продолжения анализа в Python импортируем нужные библиотеки:
 - __pandas__ предоставляет удобные инструменты для хранения данных и работе с ними
 - __psycopg2__ позволяет делать запросы к БД PostgreSQL из Python
 - __numpy__ добавляет поддержку многомерных масивов и математических функций для операций с ними
 - __matplotlib__ графическая библиотека, которая поможет нам в построении графиков по результатам анализа


In [4]:
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np
pd.set_option('display.max_rows', None) # --optional (just help to see all rows, when it's necessary)

Функция __getPurchaseCourses()__ получает данные по пользователей cо списком курсов, которые они купили ([результат выгрузки запроса](https://drive.google.com/file/d/19XfzheDC3rv9Q4DN76FAyAYcTWxMTYmn/view?usp=sharing)).

In [5]:
def getPurchaseCourses():
    query = '''
WITH purchases AS
    (
    SELECT
        DISTINCT c.user_id users,
        ci.resource_id,
        c.purchased_at
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    ORDER BY 
        c.user_id
    ),

more_one AS
    (
    SELECT
        DISTINCT c.user_id users,
        COUNT(ci.resource_id)
    FROM
        final.carts AS c
    JOIN
        final.cart_items AS ci
            ON
                c.id = ci.cart_id
    WHERE
        c.state = 'successful' AND ci.resource_type = 'Course'
    GROUP BY
        c.user_id
    HAVING
        COUNT(DISTINCT ci.resource_id) > 1
    ORDER BY 
        c.user_id
    )

SELECT
    *
FROM 
    purchases

WHERE EXISTS       
    (
        SELECT
            users
        FROM
            more_one
        WHERE
            purchases.users = more_one.users
    )
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

Результат запишем в датафрейм __purchase_df__

In [6]:
purchase_df = pd.DataFrame(getPurchaseCourses())
purchase_df.head()

,users,resource_id,purchased_at
0,51,516,2017-01-06 21:31:53.507
1,51,1099,2018-06-22 17:20:49.080
2,6117,356,2017-06-30 17:36:47.875
3,6117,357,2017-06-30 17:36:47.875
4,6117,1125,2018-08-01 05:01:45.031


Сгруппируем пользователей таким образом, чтобы напротив каждого пользователя был список id курсов, которые он приобрел. Запишем результата переменную **users_purchase_list**

In [7]:
users_purchase_list = purchase_df.groupby(['users'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
users_purchase_list.head()

,users,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [8]:
users_purchase_list.to_excel(r'users_purchase_list.xlsx')

In [7]:
len(users_purchase_list)

12656

Теперь разобьем списки **id курсов** _для каждого пользователя_ на возможные комбинации **пар курсов** с помощью библиотеки **itertools** и метода **combinations**.

In [8]:
import itertools
result_list = list()
# проход  по спискам resource_id 
for courses in users_purchase_list['resource_id']:
    # проход циклом по содержимому каждого списка с делением на пары
    for course_pair in itertools.combinations(courses, 2): 
        #результат запишем в список result_list
        result_list.append(course_pair)

In [9]:
len(result_list)

40017

In [10]:
#Удалим дубликаты, записав результат во множество result_set
result_set = set(result_list)
len(result_set)

3989

**Возвращаясь к нашему заданию**
<div class="alert alert-block alert-info">
<b>А13.1.1.3 Задание 3</b><br>Сколько различных пар курсов встречаются вместе в покупках клиентов?</div>

**Получаем ответ:**

<div class="alert alert-block alert-success">3989</div>

**Возвращаясь к нашему заданию**
<div class="alert alert-block alert-info">
<b>А13.1.1.3 Задание 3</b><br>Какая самая популярная пара курсов? В ответ запишите два числа в порядке возрастания через знак плюса без пробелов.</div>

Для этого воспользуемся библиотекой **collections**, в которой есть счетчик **Counter()**

In [11]:
import collections
c = collections.Counter()
for pairs in result_list:
    c[pairs]+=1
print(c)

Counter({(551, 566): 797, (515, 551): 417, (489, 551): 311, (523, 551): 304, (566, 794): 290, (489, 515): 286, (490, 566): 253, (490, 551): 247, (570, 752): 247, (569, 572): 216, (515, 523): 213, (553, 745): 212, (489, 523): 206, (569, 840): 204, (514, 551): 200, (516, 745): 199, (515, 566): 195, (489, 566): 188, (504, 572): 184, (572, 840): 178, (551, 552): 177, (507, 570): 172, (490, 809): 163, (489, 490): 152, (507, 752): 150, (523, 552): 144, (490, 515): 143, (551, 570): 142, (504, 569): 139, (514, 515): 139, (551, 745): 138, (514, 566): 138, (502, 551): 135, (504, 840): 135, (571, 1125): 122, (502, 566): 120, (523, 566): 120, (570, 809): 119, (752, 809): 115, (490, 523): 114, (357, 571): 112, (523, 564): 110, (551, 749): 109, (516, 553): 107, (551, 777): 107, (551, 679): 104, (356, 571): 103, (551, 564): 103, (515, 749): 103, (568, 745): 102, (356, 357): 100, (363, 511): 99, (551, 571): 98, (551, 809): 96, (502, 514): 95, (551, 794): 95, (490, 514): 94, (566, 764): 92, (490, 564):

In [12]:
#Самая популярная пара курсов
popular_pair = c.most_common(1)[0][0]
popular_pair

(551, 566)

In [11]:
#Которую заказывали:
print(c[(551, 566)],'раз')

797 раз


In [12]:
print('Самая популярная пара курсов: ', str(popular_pair[0])+'+'+str(popular_pair[1]))

Самая популярная пара курсов:  551+566


<div class="alert alert-block alert-success">551+566</div>

### Финальный шаг. Составление отчета

Теперь, когда всё почти готово, вам осталось оформить таблицу с рекомендациями для продакт-менеджера и отдела маркетинга.

Составьте таблицу с тремя столбцами:

 - Столбец 1. Курс, к которому идёт рекомендация
 - Столбец 2. Курс для рекомендации № 1 (самый популярный)
 - Столбец 3. Курс для рекомендации № 2 (второй по популярности).
 
А что делать, если одна из рекомендаций встречается слишком мало раз? В таком случае, во-первых, нужно установить минимальную границу — какое количество считать слишком малым. А во-вторых, вместо такого малопопулярного курса выводите какой-то другой курс, который, на ваш аргументированный взгляд, подходит лучше.

**Выведем уникальные id курсов для первого столбца.**

In [14]:
list_of_courses = purchase_df['resource_id'].unique()
list_of_courses

array([ 516, 1099,  356,  357, 1125,  553, 1147,  361, 1138, 1140,  551,
        745,  568,  514,  517,  566,  363,  511,  562,  563,  509, 1144,
        672,  552,  571,  513, 1141,  744,  862,  679,  750,  800,  569,
        840,  765, 1187, 1100, 1103,  502,  564,  865,  764, 1139, 1186,
        366,  367,  519,  809,  515,  912,  489,  523,  864, 1101, 1146,
        776,  671,  753,  829,  490, 1102,  803,  659,  909,  794,  518,
        907,  777,  908,  360,  813,  835,  741,  752,  814, 1115, 1116,
       1161,  863,  743,  504,  572,  810, 1124, 1128,  742, 1104,  503,
        664,  507,  570, 1185, 1198,  365,  359,  791, 1156,  362, 1184,
        911,  358, 1160,  757,  508, 1181,  755, 1145, 1188,  756,  866,
        749,  368,  364,  834, 1152,  670, 1199,  836, 1201, 1129, 1182,
        902,  837, 1200,  833,  830], dtype=int64)

In [14]:
#Количество уникальных курсов
len(list_of_courses)

126

Преобразуем наш _счетчик в словарь,_  в котором, в качестве **ключа** будет **пара курсов**, а в **качестве значения** - **популярность пары** (чем выше число, тем выше популярность)

In [15]:
dict_c = dict(c)
dict_c

{(516, 1099): 25,
 (356, 357): 100,
 (356, 1125): 44,
 (357, 1125): 52,
 (553, 1147): 16,
 (361, 1138): 40,
 (1125, 1140): 1,
 (551, 745): 138,
 (553, 745): 212,
 (551, 1138): 14,
 (553, 568): 83,
 (514, 517): 10,
 (514, 566): 138,
 (517, 566): 21,
 (363, 511): 99,
 (363, 562): 77,
 (363, 563): 33,
 (511, 562): 55,
 (511, 563): 19,
 (562, 563): 53,
 (568, 745): 102,
 (509, 553): 48,
 (509, 745): 59,
 (1125, 1144): 22,
 (509, 568): 46,
 (509, 672): 5,
 (568, 672): 4,
 (516, 552): 12,
 (356, 552): 7,
 (357, 571): 112,
 (509, 516): 35,
 (516, 568): 54,
 (513, 1141): 34,
 (571, 1125): 122,
 (551, 552): 177,
 (551, 744): 16,
 (551, 862): 8,
 (552, 744): 8,
 (552, 862): 6,
 (552, 1138): 4,
 (744, 862): 2,
 (744, 1138): 1,
 (862, 1138): 3,
 (356, 679): 8,
 (571, 745): 22,
 (571, 1099): 2,
 (745, 1099): 53,
 (509, 1099): 31,
 (568, 1099): 53,
 (517, 750): 34,
 (800, 1125): 4,
 (569, 840): 204,
 (745, 1125): 15,
 (509, 514): 8,
 (509, 551): 15,
 (514, 551): 200,
 (514, 745): 38,
 (571, 765): 83

Отсортируем наш словарь по **парам id курсов** в порядке возростания и запишем результат в словарь **sorted_pairs**

In [16]:
sorted_pairs = {k: v for k, v in sorted(dict_c.items(), key=lambda item: item[0], reverse=False)}
sorted_pairs

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

Создадим функцию **recommend**, которая будет принимать на вход **id курса** и выдавать нам список, содержащий 2 самые популярные пары курсов, в которых содержится каждый наш курс. 

In [17]:
def recommend(course):
    course_list = [] #создаем пустой список
    for i in sorted_pairs.keys(): #проходим циклом по ключам отсортированного словаря
        if i[0] == course: # если первый элемент пары курсов = id курсу, который мы подаем на вход функции
            course_list.append((i, sorted_pairs[i])) #добавляем в список кортеж, который состоит из пары курсов и количества
    sorted_course_list = sorted(course_list, key=lambda x: x[1], reverse=True)#сортируем список в порядк убывания по количеству
    return sorted_course_list[:2]#выводим только 2 значения, в которых содержаться курсы для рекомендации №1 и №2

Проверим, как работает наша функция, подав ей на вход курс с **resource_id = 356**:

In [18]:
recommend(356)

[((356, 571), 103), ((356, 357), 100)]

Мы получили _2 кортежа,_ которые содержат _2 самые популярные пары курсов_ и их _количество соответственно._

 - 1-ый элемент пары - курс, к которому мы будем рекомендовать другой курс
 - 2-ой элемент пары каждого кортежа - рекомендуемый курс №1 и №2 соответственно
 
Итого, на примере курса с **resource_id = 356**, мы получаем рекомендации двух других курсов с : **resource_id = 571** и **357** соответственно.

В идеале, результат выполнения нашей функции выдает рекомендации двух курсов, но может такое быть, что она выдаст рекомендацию только одного курса или вовсе не выдаст и с этим нам еще предстоит решить, что делать. А пока пройдемся циклом по списку всех наших курсов **list_of_courses** и оформим результаты в датафрейм.

In [19]:
recomendations_df = pd.DataFrame(columns = ['Рекомендация №1', 'Рекомендация №2'])
#создадим счетчики, чтобы потом понять сколько курсов у нас получилось 2, 1 или ниодной рекомендации
count_two, count_one, count_zero = 0, 0, 0
for i in list_of_courses:
    if len(recommend(i)) == 2: # когда функция выдает 2 рекомендации
        recomendations_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
        count_two+=1 #считаем количество курсов, получившие 2 рекомендации
    elif len(recommend(i)) == 1: # когда функция выдает 1 рекомендацию
        recomendations_df.loc[i] = [recommend(i)[0][0][1], np.nan]
        count_one+=1 #считаем количество курсов, получившие 1 рекомендацию
    else: 
        recomendations_df.loc[i] = [np.nan, np.nan] # когда функция не выдает рекомендаций
        count_zero+=1 #считаем количество курсов, не получившие рекомендации вовсе

In [20]:
recomendations_df.index.name = 'Course'  # назначим имя первому столбцу
recomendations_df

,Рекомендация №1,Рекомендация №2
Course,,
516,745,553
1099,1139,1187
356,571,357
357,571,1125
1125,1186,1144
553,745,568
1147,1187,NaN
361,551,1138
1138,1144,1139


In [21]:
print('Количество курсов, получившие 2 рекомендации:', count_two)
print('Количество курсов, получившие 1 рекомендацию:', count_one)
print('Количество курсов, не получившие рекомендации:', count_zero)

Количество курсов, получившие 2 рекомендации: 112
Количество курсов, получившие 1 рекомендацию: 5
Количество курсов, не получившие рекомендации: 9


Как мы видим, некоторые наши курсы получили 1 рекомендацию или не получили вовсе, поскольку рекомендации встречаются слишком мало раз. Для этого нам нужно придумать вариант, какие рекомендации дать для этих курсов. 

In [22]:
recomendations_df[recomendations_df['Рекомендация №2'].isnull()]

,Рекомендация №1,Рекомендация №2
Course,,
1147,1187,NaN
1187,1188,NaN
1198,NaN,NaN
1184,NaN,NaN
1160,NaN,NaN
1181,1184,NaN
1188,NaN,NaN
1152,1160,NaN
1199,NaN,NaN


Почему так вышло, что у рекомендации не ко всем курсам? Вске дело в том, что, когда мы использовали **itertools.combinations**, то пара образовывалась таким образом, что **(a, b) == (b, a)**, и выбиралась одна пара, либо **(a, b)**, либо **(b, a)**

К примеру, в пара курсов **(523, 564)** в данном случае считается  = **(564, 523)**

Поскольку в рекомендациях к курсу, мы производили расчет по **первому курсу в паре**, и выбирали **самый популярный курс, который образует с ним пару**, то остались в конце оставались те курсы, которые:
 - образовывали с другим курсом лишь 1 пару (мы давали эту единственную рекомендацию)
 - не образовывали какую-либо пару (мы не могли дать рекомендацию)

Cоздадим функцию **recommend_for_nan**, которая будет заполнять **NaN** значения, по сути она практически не отличается от функции **recommend**, единственное отличие, что внутри функции идет сравнение 2-ого значения (**i[1] == course**) в паре **sorted_pairs**, а не 1-ого (**i[0] == course**). Чтобы лучше было понятно, напомним, структуру **sorted_pairs**:

In [23]:
sorted_pairs

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

In [24]:
def recommend_for_nan(course):
    course_list_nan = [] #создаем пустой список
    for i in sorted_pairs.keys(): #проходим циклом по ключам отсортированного словаря
        if i[1] == course: # если первый элемент пары курсов = id курсу, который мы подаем на вход функции
            course_list_nan.append((i, sorted_pairs[i])) #добавляем в список кортеж, который состоит из пары курсов и количества
    sorted_course_list = sorted(course_list_nan, key=lambda x: x[1], reverse=True)#сортируем список в порядк убывания по количеству
    return sorted_course_list[:2]#выводим все значения, среди которых потом выберем недоставющую вторую рекомендацию для курса или обе рекомендации

Заполним наш новый датафрейм **recomendations_df_final** рекомендациями:

 - **2** рекомендации через функцию **recommend**
 - **1** рекомендация через функцию **recommend** + **1** рекомендация через функцию **recommend_for_nan**
 - **2** рекомендации через функцию **recommend_for_nan**

In [25]:
recomendations_df_final= pd.DataFrame(columns = ['Рекомендация №1', 'Рекомендация №2'])

for i in list_of_courses:
    if len(recommend(i)) == 2: # когда функция выдает 2 рекомендации
        recomendations_df_final.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
    #когда функция recommend выдает 1 рекомендацию, то используем в качестве второй рекомендации результат выполнения второй функции recommend_for_nan 
    elif len(recommend(i)) == 1: 
        recomendations_df_final.loc[i] = [recommend(i)[0][0][1], recommend_for_nan(i)[1][0][0]]
    # когда функция recommend не выдает рекоменаций, то используем в качестве  1-ой и 2-ой рекомендаций результат выполнения второй функции recommend_for_nan
    else: 
        recomendations_df_final.loc[i] = [recommend_for_nan(i)[0][0][0], recommend_for_nan(i)[1][0][0]]


Чтобы лучше понять вложенность списков в функциях (например, **recommend(i)[0][0][1]**), покажем результат выполнения функций, ограничив вывод до **5** значений:

In [26]:
for i in list_of_courses[:5]:
    print(recommend(i))

[((516, 745), 199), ((516, 553), 107)]
[((1099, 1139), 20), ((1099, 1187), 16)]
[((356, 571), 103), ((356, 357), 100)]
[((357, 571), 112), ((357, 1125), 52)]
[((1125, 1186), 27), ((1125, 1144), 22)]


In [27]:
for i in list_of_courses[:5]:
    print(recommend_for_nan(i))

[((515, 516), 42), ((509, 516), 35)]
[((568, 1099), 53), ((745, 1099), 53)]
[]
[((356, 357), 100)]
[((571, 1125), 122), ((912, 1125), 55)]


In [28]:
recomendations_df_final.index.name = 'Course'

Проверим остались ли у нас **NaN** значения в нашем финальном датафрейме с рекоменлациями:

In [29]:
recomendations_df_final[recomendations_df_final['Рекомендация №2'].isnull()]

,Рекомендация №1,Рекомендация №2
Course,,


In [30]:
recomendations_df_final[recomendations_df_final['Рекомендация №1'].isnull()]

,Рекомендация №1,Рекомендация №2
Course,,


**Как видим, все рекомендации даны, теперь можем вывести финальную таблицу с рекомендациями:**

In [31]:
recomendations_df_final.index.name = 'Course' # назначим имя первому столбцу
recomendations_df_final

,Рекомендация №1,Рекомендация №2
Course,,
516,745,553
1099,1139,1187
356,571,357
357,571,1125
1125,1186,1144
553,745,568
1147,1187,553
361,551,1138
1138,1144,1139


**Экспортируем её в xlsx файл:**

In [32]:
recomendations_df_final.to_excel(r'recommendation_table.xlsx')

**Файл можно посмотреть локально или скачать по** [ссылке](https://drive.google.com/file/d/10GAFQflyoUyS0Xgy6-xUgdldgNoHCec5/view?usp=sharing)

<hr>

### Это ещё не всё! Тестирование гипотезы

Хороший аналитик не может просто взять, отдать табличку с рекомендациями и спать спокойно. Хороший аналитик после внедрения предложенной системы обязательно включится в работу по тестированию гипотезы. 

Итак, спустя месяц на сайте реализована новая функциональность с предложением добавить в корзину второй подходящий курс. Ваши коллеги хотят оценить эффективность этой функции и качество подбора рекомендаций.

Для этого запускается **сплит-тест**, где все клиенты случайным образом делятся на контрольную и тестовую группу. Тестовой группе показываются рекомендации, а контрольной — нет.

До реализации рекомендаций средняя конверсия в покупку второго курса была **3,2%**. Вы ожидаете, что ввод рекомендаций сможет подрастить её до **4%**.

<div class="alert alert-block alert-info">
<b>А13.1.3.1 Задание 1</b><br>Определите минимальный размер выборки для проведения теста при уровне достоверности 95 % и статистической мощности 80 %. Округлите ответ до сотен.</div>